<h1 style="background-color:#FF7733;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
📐 תהליך הסבת שכבת חזיתות בתהליך לבנק"ל מודרני 📐 

In [ ]:
print('13. Fronts In-Process:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

In [ ]:
print('  13.a Reading inputs')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
RawData_folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

today = dt.date.today()
date_name =  str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name

<h1 style="text-align:right;font-size:100%">
TEMP_ID הרצת עיבוד מקדים ליצירת שדה מקשר

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Fronts_InProcess-preprocess.bat"]);

<h1 style="text-align:right;font-size:100%">
שכבת חזיתות בתהליך קיימת

In [ ]:
fronts_inprocess = pd.read_csv(RawData_folder + 'CADSDE/Lines_InProcess.csv', engine = 'c',
                               usecols = ['TEMP_ID', 'TALAR_NUM', 'TALAR_YEAR', 'ENTITY_STATE', 'FRONT_LEGAL_LENGTH', 'CUR_RADIUS'])
print('     input 1/2 read')

In [ ]:
cpb_layer = gpd.read_file(product_path,
                          layer = 'CadasterProcessBorders',
                          include_fields = ['CPBUniqueID', 'ProcessName'],
                          ignore_geometry = True)

print('     input 2/2 read')

In [ ]:
del [RawData_folder, today, date_name, product_path]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  13.b Cleaning data')

<h1 style="text-align:right;font-size:100%">
חזיתות בעלות שטח רשום ריק יקבלו את הערך 0

In [ ]:
fronts_inprocess['FRONT_LEGAL_LENGTH'] = fronts_inprocess['FRONT_LEGAL_LENGTH'].fillna(0.0)

In [ ]:
print('  13.c Data conversion')

<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות אורך מחושב ורדיוס

In [ ]:
fronts_inprocess.rename(columns = {'FRONT_LEGAL_LENGTH' : 'LegalLength',
                                   'CUR_RADIUS'         : 'Radius'},
                        inplace = True)

<h1 style="text-align:right;font-size:100%">
סוג החזית יקבע לפי קיום של רדיוס

In [ ]:
straight = 1
curved   = 2

fronts_inprocess['LineType'] = np.where(fronts_inprocess['Radius'].isna(), straight, curved)

del [straight, curved]

<h1 style="text-align:right;font-size:100%">
ENTITY_STATE סטאטוס החזית תקבע לפי שדה
    <h1 style="text-align:right;font-size:100%">
(Null ישנם מקרים בהם השדה ישאר)

In [ ]:
old_front = 1
new_front = 2

fronts_inprocess['LineStatus'] = np.where(fronts_inprocess['ENTITY_STATE'] == 0, old_front, new_front)

fronts_inprocess.drop(columns = 'ENTITY_STATE', inplace = True)

del [old_front, new_front]

<h1 style="text-align:right;font-size:100%">
איחוד עם שכבת גבולות תהליכי קדסטר לקבלת מזהה של תהליך יוצר

In [ ]:
fronts_inprocess[['TALAR_NUM', 'TALAR_YEAR']] = fronts_inprocess[['TALAR_NUM', 'TALAR_YEAR']].fillna(0)
fronts_inprocess[['TALAR_NUM', 'TALAR_YEAR']] = fronts_inprocess[['TALAR_NUM', 'TALAR_YEAR']].astype('Int32')
fronts_inprocess['ProcessName'] = fronts_inprocess['TALAR_NUM'].astype(str) + '/' + fronts_inprocess['TALAR_YEAR'].astype(str)

fronts_inprocess = fronts_inprocess.merge(right = cpb_layer, on = 'ProcessName', how = 'left')
fronts_inprocess['CPBUniqueID'] = fronts_inprocess['CPBUniqueID'].fillna(0)
fronts_inprocess['CPBUniqueID'] = fronts_inprocess['CPBUniqueID'].astype('Int64')

fronts_inprocess.drop(columns = ['TALAR_NUM', 'TALAR_YEAR', 'ProcessName'], inplace = True)

<h1 style="text-align:right;font-size:100%">
יצירת מספר מזהה לכל חזית בתהליך

In [ ]:
fronts_inprocess['FrontUniqueID'] = range(1, 1 + len(fronts_inprocess))

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
 print('  13.d Defining new Layer')

<h1 style="text-align:right;font-size:100%">
הגדרת סוגי שדות נתונים

In [ ]:
fronts_inprocess = fronts_inprocess.astype({'FrontUniqueID'       : int,
                                            'LineType'            : int,
                                            'LineStatus'          : 'Int64',
                                            'Radius'              : float,
                                            'LegalLength'         : float,
                                            'CPBUniqueID'         : 'Int64' ,
                                            'TEMP_ID'             : int})

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  13.e Exporting results to product folder')

In [ ]:
csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\InProcessFronts_data.csv'

fronts_inprocess.to_csv(csv_dir, index = False)

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Fronts In-Process-Save as Feature Class.bat"]);